In [ ]:
from tqdm import tqdm

In [ ]:
from datasets import load_dataset

dataset = load_dataset("arbml/tashkeela")

In [ ]:
def count_words(text):
    return len(text.split())

In [ ]:
def unique_characters(text1, text2):
    # Convert texts into sets of characters
    set_text1 = set(text1)
    set_text2 = set(text2)

    # Find unique characters in each set
    unique_in_text1 = set_text1.difference(set_text2)

    return unique_in_text1

In [ ]:
unique_chars = set()  # Initialize an empty set to store unique characters
n_words = 0  # Initialize a counter for the number of words

for example in tqdm(dataset["train"]):
    unique_text1 = unique_characters(example["diacratized"], example["text"])
    unique_chars.update(unique_text1)
    n_words += count_words(example["text"])

In [ ]:
print(f"Number of words: {n_words}")

In [ ]:
# Print Unicode encoding in hexadecimal
for char in sorted(unique_chars):
    print(f"Character: {char}, Unicode hexadecimal: {ord(char):x}")

In [ ]:
import re


def strip_diacritics(text):
    pattern = re.compile("[\u064B-\u0652]")

    stripped_text = pattern.sub("", text)

    return stripped_text


# Example usage:
text = example["diacratized"]
stripped_text = strip_diacritics(text)
print("Stripped text:", stripped_text)

In [ ]:
# check example["diacratized"] and example["text"] are equal after stripping diacritics
stripped_text = strip_diacritics(example["diacratized"])
stripped_text == example["text"]

In [ ]:
original_line = example["diacratized"].split()

# Iterate over each word pair
for original_word in original_line:
    # Calculate the size of the resulting sentence if we add the next word
    print(original_word)
    print(original_word.encode("utf-8"))
    print(len(original_word.encode("utf-8")))
    break

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

# Tokenize the text
encoded = tokenizer.batch_encode_plus(
    [original_word], max_length=512, padding="max_length", truncation=True, return_tensors="pt"
).input_ids

In [ ]:
tokenizer.encode(
    example["diacratized"],
    truncation=True,
    padding="max_length",
    max_length=700,
    return_tensors="pt",
)

In [ ]:
encoded[encoded == tokenizer.pad_token_id] = -100

In [ ]:
encoded

In [ ]:
tokenizer.decode(encoded[0], skip_special_tokens=True)

In [ ]:
def strip_diacritics(text):
    pattern = re.compile("[\u064b-\u0652]")

    stripped_text = pattern.sub("", text)

    return stripped_text


def tokenize(example, max_len):
    target_line = example["diacratized"]

    target_line = target_line.split()

    new_target_line = ""
    byte_count = 0

    # Iterate over each word pair
    for target_word in target_line:
        # Calculate the size of the resulting sentence if we add the next word
        next_byte_count = byte_count + len(target_word.encode("utf-8"))
        if new_target_line:
            next_byte_count += len(" ")

        # Check if adding the next word will exceed the byte limit
        if next_byte_count > max_len:
            break

        # Add preceding space only if it's not the first word
        if new_target_line:
            new_target_line += " "
        new_target_line += target_word

        byte_count = next_byte_count

    example["diacratized"] = new_target_line
    example["text"] = strip_diacritics(new_target_line)

    return example

In [ ]:
dataset = load_dataset("arbml/tashkeelav2")
tokenized_dataset = dataset.map(tokenize, fn_kwargs={"max_len": 512}, num_proc=6)
tokenized_dataset.save_to_disk("data")

In [ ]:
def strip_diacritics(text):
    pattern = re.compile("[\u064b-\u0652]")

    stripped_text = pattern.sub("", text)

    return stripped_text

In [ ]:
from datasets import load_from_disk

dataset = load_from_disk("data", keep_in_memory=True)

for example in dataset["test"]:
    stripped_text = strip_diacritics(example["diacratized"])
    # assert len(example["diacratized"].split()) == len(example["text"].split())
    assert len(example["diacratized"].encode("utf-8")) <= 512
    if stripped_text != example["text"]:
        print(stripped_text)
        print(example["text"])
        break

In [ ]:
from datasets import load_dataset

dataset = load_dataset("arbml/tashkeelav2")
dataset = dataset.map(tokenize, fn_kwargs={"max_len": 100})
dataset.save_to_disk("data")

In [ ]:
from datasets import load_from_disk
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")
dataset = load_from_disk("data", keep_in_memory=True)
split = "test"
max_len = 512

dataloader = DataLoader(dataset, batch_size=1, num_workers=0)

In [ ]:
from tqdm import tqdm

for batch in tqdm(dataloader):
    if batch["input_ids"].shape[1] != max_len:
        print("input_ids")
        print(batch["input_ids"].shape)
        break
    if batch["attention_mask"].shape[1] != max_len:
        print("attention_mask")
        print(batch["attention_mask"].shape)
        break
    if batch["target_ids"].shape[1] != max_len:
        print("target_ids")
        print(batch["target_ids"].shape)
        break

In [ ]:
decoded = tokenizer.decode(batch["target_ids"][0], skip_special_tokens=True)
len(decoded.encode("utf-8"))